In [1]:
import os
import sys


sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "src")))

In [2]:
import torch


env_name = "button-press-topdown-v2"
exp_name = "CUDA-test"
pair_algo = "ternary-500"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAJECTORY_LENGTH = 25


In [3]:
from policy_learning.change_reward_pt import change_reward_and_save_pt
from reward_learning.train_model import train_reward_model


train_reward_model(
    env_name=env_name,
    exp_name=exp_name,
    pair_algo=pair_algo,
    reward_model_algo="PT",
    reward_model_tag="00",
    num_epoch=100,
)

change_reward_and_save_pt(
    env_name=env_name,
    exp_name=exp_name,
    pair_algo=pair_algo,
)

Loaded pair/button-press-topdown-v2/CUDA-test/train/ternary-500.npz dataset
Processed data with 500 samples
Loaded pair/button-press-topdown-v2/CUDA-test/val/ternary-500.npz dataset
Processed data with 500 samples
obs_dim: 39 act_dim: 4
Skipping model initialization because already exists
Model loaded from model/button-press-topdown-v2/CUDA-test/reward/ternary-500/PT_00.pth


Relabel Trajectory:   1%|          | 5/600 [00:16<33:33,  3.38s/it]


KeyboardInterrupt: 